# -*- coding: utf# -*- coding: utf-8 -*-
"""

#### Necesidad de financiamiento de inversiones por caída de Recursos Determinados.
#### en pro de asegurar continuidad

# Criterios mínimos:
    # Continuidad: sobre la base de los avances en el 2023 y 2022
    # Asegurar el pago de contratos ya registrados en SEACE
    # Dimensión del costo
    
#%%-8 -*-
"""
"""

#### Necesidad de financiamiento de inversiones por caída de Recursos Determinados.
#### en pro de asegurar continuidad

# Criterios mínimos:
    # Continuidad: sobre la base de los avances en el 2023 y 2022
    # Asegurar el pago de contratos ya registrados en SEACE
    # Dimensión del costo
    
#%%

In [1]:

import pandas as pd
import numpy as np
import os

def postmerge(df):
    replace_list = [i for i in df.columns if (i.endswith('_x'))]
    replace_list = [i[:-2] for i in replace_list]
    for i in replace_list:
        df[i] = df[i+'_x'].fillna(df[i+'_y'])
    df.drop([i for i in df.columns if (i.endswith('_x'))|(i.endswith('_y'))], inplace=True, axis=1)
    return df

#%%

In [2]:


# Sec ejec
bd_secejec = pd.read_excel(os.path.join(r'C:\Users\jchamba\Documents\Jannely\BASES', 'SEC_EJEC_UBIGEO.xlsx'))

# Ingresos
bd_ingresos = pd.read_excel(os.path.join(r'C:\Users\jchamba\Documents\Jannely\Proyecto 4\Bases', 'Base1 (3).xlsx'), sheet_name='Lista preliminar 1', skiprows=3)

# SIAF INGRESOS
bd_siafing = pd.read_excel(os.path.join(r'Z:\Ingresos\0. Data\1. SIAF Presupuesto Ingresos\2024', 'RecIngresos_2024_GNGRGLMN_01022024.xlsx'))

# SIAF
bd_siaf24 = pd.read_csv(os.path.join(r'Y:\Data_CSV\SIAFGasto2024', 'SeguimientoGasto_2024_29022024.csv'))
bd_siaf23 = pd.read_csv(os.path.join(r'C:\Proyecto 4\Bases', 'SeguimientoGastoGK_2023_26012024.csv'))
bd_siaf22 = pd.read_csv(os.path.join(r'C:\Proyecto 4\Bases', 'SeguimientoGastoGK_2022.csv'))

# Invierte.pe
bd_invpe = pd.read_csv(os.path.join(r'Y:\Data_CSV\BancoInv', 'InviertePe_26022024.csv'))

# SEACE
bd_contrato = pd.read_csv(os.path.join(r'C:\Proyecto 4\Bases', 'Contratos_22-23-24_20240129.csv'))
bd_convoca = pd.read_csv(os.path.join(r'C:\Proyecto 4\Bases', 'Convocatorias_23-24_20240129.csv'))

# Reducciones de marco
bd_mod_gr = pd.read_excel(os.path.join(r'C:\Users\jchamba\Documents\Jannely\Proyecto 4\Bases', 'gr_reducciones_01022024.xlsx'))
bd_mod_gl = pd.read_excel(os.path.join(r'C:\Users\jchamba\Documents\Jannely\Proyecto 4\Bases', 'gl_reducciones_01022024.xlsx'))

bd_mod_gr7 = pd.read_excel(os.path.join(r'C:\Users\jchamba\Documents\Jannely\Proyecto 4\Bases', 'gr_reducciones_29022024.xlsx'))
bd_mod_gl7 = pd.read_excel(os.path.join(r'C:\Users\jchamba\Documents\Jannely\Proyecto 4\Bases', 'gl_reducciones_29022024.xlsx'))

# Reducciones anteriores
bd_mod_ant = pd.read_excel(os.path.join(r'C:\Users\jchamba\Documents\Jannely\Proyecto 4\Bases', 'BD_rd31eval_Vresum_6GR.xlsx'))

C:\Users\jchamba\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\jchamba\AppData\Local\Temp\ipykernel_5696\2929970576.py:16: DtypeWarning: Columns (17,51) have mixed types. Specify dtype option on import or set low_memory=False.
  bd_invpe = pd.read_csv(os.path.join(r'Y:\1) NUEVO compartido DAPT\Financiamiento_Data\Data_CSV\BancoInv', 'InviertePe_26022024.csv'))


In [3]:
#%% Por sec ejec
secejec = bd_secejec.copy()
secejec['SEC_EJEC'] = secejec['SEC_EJEC'].astype(int).astype(str)
secejec['UBIGEO'] = secejec['UBIGEO'].fillna(0).astype(int).astype(str)
secejec['UBIGEO'] = np.where(secejec['UBIGEO'].str.len()<6, '0' + secejec['UBIGEO'], secejec['UBIGEO'])
secejec['PROVINCIA'] = secejec['PROVINCIA'].fillna('')
secejec['ID_PLIEGO_EJEC_GR'] = secejec['PLIEGO_SIAF'].str.split('. ',n=1).str[0] + secejec['EJECUTORA'].str.split('. ',n=1).str[0]

#%% Reducciones de marco
bajomarco = pd.concat([bd_mod_gr, bd_mod_gl], ignore_index=True)
#bajomarco = bajomarco[bajomarco['TIPO_MODIF'].astype(int).astype(str) == '7']

bajomarco = bajomarco[bajomarco['RUBRO'].str[0:2] == '18']
print(bajomarco.MONTO_ANULACION.sum())

#bajomarco['ID_RD31_MODIF'] = np.where((bajomarco['DES_TEXTO'].str.contains('31|SEGUNDA|DCT')), 'SI', 'NO') 

bajomarco = bajomarco[~((bajomarco['DES_TEXTO'].str.contains('SALDO|BALANCE'))&(~(bajomarco['DES_TEXTO'].str.contains('31'))))]

bajomarco['ID_RD31_MODIF'] = 'SI'

# (bd_notas_final['ACT_OBRA_ACCINV'].str.contains('MANTENIMIENTO'))

bajomarco['CODIGO_UNICO'] = bajomarco['PROD_PROY'].str.split('. ',n=1).str[0]
bajomarco['SEC_EJEC'] = bajomarco['SEC_EJEC'].astype(int).astype(str)
bajomarco.rename(columns={'MONTO_ANULACION':'REDUCCION_MARCO'}, inplace=True)


bajomarco = bajomarco.groupby(['SEC_EJEC', 'CODIGO_UNICO', 'ID_RD31_MODIF']).agg({'REDUCCION_MARCO':'sum'}).reset_index()
bajomarco = bajomarco.groupby(['SEC_EJEC', 'CODIGO_UNICO']).agg({'REDUCCION_MARCO':'sum',
                                                                 'ID_RD31_MODIF':'-'.join}).reset_index()

print(bajomarco[['SEC_EJEC', 'CODIGO_UNICO']].duplicated().sum())
bajomarco_0102 = bajomarco.copy()

###############################################################################
bajomarco = pd.concat([bd_mod_gr7, bd_mod_gl7], ignore_index=True)
#bajomarco = bajomarco[bajomarco['TIPO_MODIF'].astype(int).astype(str) == '7']

bajomarco['ID_FECHAOK'] = np.where(bajomarco['FECHA_APRUEBA'].str[3:5] == '01', 'SI', 'NO') # mes enero


bajomarco = bajomarco[bajomarco['RUBRO'].str[0:2] == '18'] # rubro 18
print(bajomarco.MONTO_ANULACION.sum())

#bajomarco['ID_RD31_MODIF'] = np.where((bajomarco['DES_TEXTO'].str.contains('31|SEGUNDA|DCT|COMPLEMENTARIA TRANSITORIA')), 'SI', 'NO') 
bajomarco['ID_RD31_MODIF'] = 'SI'
# (bd_notas_final['ACT_OBRA_ACCINV'].str.contains('MANTENIMIENTO'))

bajomarco['CODIGO_UNICO'] = bajomarco['PROD_PROY'].str.split('. ',n=1).str[0]
bajomarco['SEC_EJEC'] = bajomarco['SEC_EJEC'].astype(int).astype(str)
bajomarco.rename(columns={'MONTO_ANULACION':'REDUCCION_MARCO'}, inplace=True)


bajomarco = bajomarco.groupby(['SEC_EJEC', 'CODIGO_UNICO', 'ID_RD31_MODIF', 'ID_FECHAOK']).agg({'REDUCCION_MARCO':'sum'}).reset_index()

bajomarco = bajomarco.groupby(['SEC_EJEC', 'CODIGO_UNICO', 'ID_RD31_MODIF']).agg({'REDUCCION_MARCO':'sum',
                                                                                  'ID_FECHAOK':'-'.join}).reset_index()

bajomarco = bajomarco.groupby(['SEC_EJEC', 'CODIGO_UNICO']).agg({'REDUCCION_MARCO':'sum',
                                                                 'ID_RD31_MODIF':'-'.join,
                                                                 'ID_FECHAOK':'-'.join}).reset_index()

print(bajomarco[['SEC_EJEC', 'CODIGO_UNICO']].duplicated().sum())

#%% Selección de municipalidades
ingresos = bd_ingresos.copy()
ingresos = ingresos[ingresos['TOTAL']<0]
ingresos['UBIGEO'] = ingresos['UBIGEO'].astype(int).astype(str)
ingresos['UBIGEO'] = np.where(ingresos['UBIGEO'].str.len()<6, '0' + ingresos['UBIGEO'], ingresos['UBIGEO'])
ingresos['Cuartil 1'] = ingresos['Cuartil 1'].fillna(0).astype(int).astype(str)
ingresos = ingresos[['UBIGEO', 'Cuartil 1', 'TOTAL']]
ingresos.rename(columns={'TOTAL':'CAIDA_RD031'}, inplace=True)

#%% --------------------------------------------------------------------------- DATOS DE SIAF INGRESOS
siaf_ing = bd_siafing.copy()
siaf_ing['SEC_EJEC'] = siaf_ing['SEC_EJEC'].astype(int).astype(str)
siaf_ing.rename(columns={'NIVEL_GOBIERNO':'NIVEL_GOB'}, inplace=True)
siaf_ing = siaf_ing[siaf_ing['NIVEL_GOB'].str[0] != '1']
siaf_ing = siaf_ing.merge(secejec, on=['SEC_EJEC'], how='left', validate='m:1')
siaf_ing = postmerge(siaf_ing)

siaf_ing = siaf_ing.groupby(['SEC_EJEC', 'UBIGEO', 'RUBRO', 'GENERICA']).agg({'NIVEL_GOB':'first', 'SECTOR':'first', 'PLIEGO':'first', 'EJECUTORA':'first',
                                                                  'DEPARTAMENTO':'first', 'PROVINCIA':'first',
                                                                  'PIA':'sum', 'PIM':'sum'}).reset_index()

siaf_ing = siaf_ing[siaf_ing['RUBRO'].str[0:2] == '18']
siaf_ing = siaf_ing[siaf_ing['GENERICA'].str[0].isin({'4', '9', '5'})]
siaf_ing['GENERICA'] = siaf_ing['GENERICA'].str[0]
siaf_ing.rename(columns={'PIA':'ING_PIA_R18_GG', 'PIM':'ING_PIM_R18_GG'}, inplace=True)
siaf_ing = siaf_ing.pivot(index=['UBIGEO', 'NIVEL_GOB', 'SECTOR', 'PLIEGO', 'EJECUTORA', 'DEPARTAMENTO', 'PROVINCIA', 'SEC_EJEC'], columns='GENERICA', values=['ING_PIA_R18_GG', 'ING_PIM_R18_GG']).reset_index().rename_axis(None)
siaf_ing.columns = [f'{i}{j}' for i, j in siaf_ing.columns]

siaf_ing = siaf_ing.merge(ingresos, on=['UBIGEO'], how='right', validate='m:1')
siaf_ing['NUEVO_INGRESO_R18'] = siaf_ing[['ING_PIA_R18_GG4', 'ING_PIA_R18_GG5']].sum(axis=1)
print(siaf_ing.SEC_EJEC.duplicated().sum())

siaf_ing['DUP'] = 1
siaf_ing['DUP'] = siaf_ing.groupby(['SEC_EJEC'])['DUP'].transform('count')

#%% --------------------------------------------------------------------------- DATOS DE SIAF

siaf22 = bd_siaf22.copy()
siaf22['SEC_EJEC'] = siaf22['SEC_EJEC'].astype(int).astype(str)
siaf22['CODIGO_UNICO'] = siaf22['CODIGO_UNICO'].astype(int).astype(str)
siaf22 = siaf22[siaf22['CODIGO_UNICO'].str[0] == '2'] # base de solo proyectos
siaf22['RUBRO'] = np.where(siaf22['RUBRO'].str[0:2] == '18', 'R18', 
                           np.where(siaf22['RUBRO'].str[0:2] == '00', 'RO', 'OTR'))

siaf22 =  siaf22.groupby(['SEC_EJEC', 'RUBRO', 'CODIGO_UNICO']).agg({'NIVEL_GOB':'first', 'SECTOR':'first', 'PLIEGO':'first',
                                                                               'EJECUTORA':'first', 
                                                                               'PRODUCTO_PROYECTO':'first', 
                                                                               'PIA':'sum', 'PIM':'sum', 'CERTIFICADO':'sum', 
                                                                               'COMPROMISO_ANUAL':'sum', 'TOTAL_DEVENGADO':'sum'}).reset_index()
siaf22.rename(columns={'PIA':'PIA_2022', 'PIM':'PIM_2022', 'CERTIFICADO':'CERTIF_2022', 'COMPROMISO_ANUAL':'COMPROM_2022',
                       'TOTAL_DEVENGADO':'DEVENG_2022'}, inplace=True)

siaf23 = bd_siaf23.copy()
siaf23['SEC_EJEC'] = siaf23['SEC_EJEC'].astype(int).astype(str)
siaf23['CODIGO_UNICO'] = siaf23['CODIGO_UNICO'].astype(int).astype(str)
siaf23 = siaf23[siaf23['CODIGO_UNICO'].str[0] == '2'] # base de solo proyectos
siaf23['RUBRO'] = np.where(siaf23['RUBRO'].str[0:2] == '18', 'R18', 
                           np.where(siaf23['RUBRO'].str[0:2] == '00', 'RO', 'OTR'))

siaf23 =  siaf23.groupby(['SEC_EJEC', 'RUBRO', 'CODIGO_UNICO']).agg({'NIVEL_GOB':'first', 'SECTOR':'first', 'PLIEGO':'first',
                                                                               'EJECUTORA':'first', 
                                                                               'PRODUCTO_PROYECTO':'first', 
                                                                               'PIA':'sum', 'PIM':'sum', 'CERTIFICADO':'sum', 
                                                                               'COMPROMISO_ANUAL':'sum', 'TOTAL_DEVENGADO':'sum'}).reset_index()
siaf23.rename(columns={'PIA':'PIA_2023', 'PIM':'PIM_2023', 'CERTIFICADO':'CERTIF_2023', 'COMPROMISO_ANUAL':'COMPROM_2023',
                       'TOTAL_DEVENGADO':'DEVENG_2023'}, inplace=True)


922581347
0
1030858642
0
0


In [4]:
bd_siaf24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149522 entries, 0 to 1149521
Data columns (total 31 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   TIPO_PROD_PROY         1149522 non-null  object 
 1   SEC_EJEC               1149522 non-null  int64  
 2   COD_PRODUCTO_PROYECTO  1149522 non-null  int64  
 3   COD_ACT_OBRA           1149522 non-null  int64  
 4   COD_FUENTE_RUBRO       1149522 non-null  object 
 5   COD_PROG_PPTAL         1149522 non-null  object 
 6   COD_CADENA_FUNCIONAL   1149522 non-null  object 
 7   COD_CADENA_GASTO       1149522 non-null  object 
 8   COD_FINALIDAD          1149522 non-null  object 
 9   CANTIDAD_META          1149522 non-null  float64
 10  COD_UNIDAD_MEDIDA      1149522 non-null  object 
 11  UBIGEO                 1149522 non-null  object 
 12  PIA                    1149522 non-null  int64  
 13  PIM                    1149522 non-null  int64  
 14  CERTIFICADO       

In [5]:
siaf24 = bd_siaf24.copy()
siaf24['SEC_EJEC'] = siaf24['SEC_EJEC'].astype(int).astype(str)
siaf24['CODIGO_UNICO'] = siaf24['COD_PRODUCTO_PROYECTO'].astype(int).astype(str)
siaf24 = siaf24[siaf24['CODIGO_UNICO'].str[0] == '2'] # base de solo proyectos
siaf24['RUBRO'] = np.where(siaf24['COD_FUENTE_RUBRO'].str[-2:] == 'FR.5.18', 'R18', 
                           np.where(siaf24['COD_FUENTE_RUBRO'].str[-2] == '00', 'RO', 'OTR'))

siaf24 =  siaf24.groupby(['SEC_EJEC', 'RUBRO', 'CODIGO_UNICO']).agg({'COD_ID':'first', 
                                                                     'PIA':'sum', 'PIM':'sum', 'CERTIFICADO':'sum', 
                                                                     'COMPROMISO_ANUAL':'sum', 'TOTAL_DEVENGADO':'sum'}).reset_index()         
siaf24.rename(columns={'PIA':'PIA_2024', 'PIM':'PIM_2024', 'CERTIFICADO':'CERTIF_2024', 'COMPROMISO_ANUAL':'COMPROM_2024',
                       'TOTAL_DEVENGADO':'DEVENG_2024'}, inplace=True)

#------------------------------------------------------------------------------ Match
siaf = pd.merge(siaf22, siaf23, on=['SEC_EJEC', 'RUBRO', 'CODIGO_UNICO'], how='outer', validate='1:1')
siaf = postmerge(siaf)
siaf = pd.merge(siaf, siaf24, on=['SEC_EJEC', 'RUBRO', 'CODIGO_UNICO'], how='outer', validate='1:1')
siaf = postmerge(siaf)

siaf = siaf.merge(secejec[['SEC_EJEC']], on='SEC_EJEC', how='left', validate='m:1')

#------------------------------------------------------------------------------ Conversión a una base wide
siaf_nombrescui = siaf[['CODIGO_UNICO', 'PRODUCTO_PROYECTO']].drop_duplicates(subset=['CODIGO_UNICO'])
siaf = siaf.pivot(index=['SEC_EJEC', 'CODIGO_UNICO'], 
                  columns='RUBRO', values=['PIA_2022', 'PIM_2022', 'CERTIF_2022', 'COMPROM_2022', 'DEVENG_2022', 
                                           'PIA_2023', 'PIM_2023', 'CERTIF_2023', 'COMPROM_2023', 'DEVENG_2023', 
                                           'PIA_2024', 'PIM_2024', 'CERTIF_2024', 'COMPROM_2024', 'DEVENG_2024']).reset_index().rename_axis(None)
siaf.columns = [f'{i}{j}' for i, j in siaf.columns]
siaf = siaf.merge(siaf_nombrescui, on=['CODIGO_UNICO'], how='left', validate='m:1')

for y in ['PIA_', 'PIM_', 'CERTIF_', 'COMPROM_', 'DEVENG_']:
    for n in list(range(2022,2025)):
        siaf[y + str(n)] = siaf[[i for i in siaf.columns if i.startswith(y + str(n))]].sum(axis=1)

#%% --------------------------------------------------------------------------- Inf. de INVPE
invpe = bd_invpe.copy()
invpe['CODIGO_UNICO'] = invpe['CODIGO_UNICO'].astype(int).astype(str)
invpe = invpe[['CODIGO_UNICO', 'TIPO_PROYECTO', 'SITUACION', 'ESTADO', 'CERRADO', 'EXP_TCO', 'ET_VIGENTE', 'COSTO_ACTUAL_BCO',
               'DEV_ACUM_AL2023']].drop_duplicates(subset=['CODIGO_UNICO'])

#%%---------------------------------------------------------------------------- Contratos 
contrata2 = bd_contrato.copy()
contrata2 = contrata2[['PERIODO_SEACE', 'SEC_EJEC', 'CODIGO_UNICO', 'FECHA_FIN_CONTRATO_ANO', 'OBJETOCONTRACTUAL']]
contrata2['SEC_EJEC'] = contrata2['SEC_EJEC'].astype(int).astype(str)
contrata2['CODIGO_UNICO'] = contrata2['CODIGO_UNICO'].astype(int).astype(str)

contrata2['OBJETOCONTRACTUAL'] = contrata2['OBJETOCONTRACTUAL'].fillna('NA')
contrata2['FECHA_FIN_CONTRATO_ANO'] = contrata2['FECHA_FIN_CONTRATO_ANO'].fillna(0)

contrata2['OBJETOCONTRACTUAL'] = np.where(contrata2['OBJETOCONTRACTUAL']=='Consultoría de Obra', 'CONSULT',
                                          np.where(contrata2['OBJETOCONTRACTUAL']=='NA', 'NOINFO', 'OTROS'))

contrata2.rename(columns={'FECHA_FIN_CONTRATO_ANO':'FINCONT_'}, inplace=True)
contrata2 = contrata2.groupby(['PERIODO_SEACE', 'SEC_EJEC', 'CODIGO_UNICO', 'OBJETOCONTRACTUAL']).agg({'FINCONT_':'max'}).reset_index()
contrata2['PERIODO_OBJETO'] = contrata2['PERIODO_SEACE'] + '-' + contrata2['OBJETOCONTRACTUAL']
contrata2 = contrata2.pivot(index=['SEC_EJEC', 'CODIGO_UNICO'], columns='PERIODO_OBJETO', values=['FINCONT_']).reset_index().rename_axis(None)
contrata2.columns = [f'{i}{j}' for i, j in contrata2.columns]

contrata2['FINCONT_MAX_TOTAL'] = contrata2[[i for i in contrata2.columns if i.startswith('FINCONT')]].max(axis=1)
contrata2['FINCONT_MAX_OTROS'] = contrata2[['FINCONT_22_23-OTROS', 'FINCONT_23_24-OTROS']].max(axis=1)

#------------------------------------------------------------------------------ Contrato y convocatorias
contrato = bd_contrato.copy()
contrato['SEC_EJEC'] = contrato['SEC_EJEC'].astype(int).astype(str)
contrato['CODIGO_UNICO'] = contrato['CODIGO_UNICO'].astype(int).astype(str)
contrato = contrato[['PERIODO_SEACE', 'SEC_EJEC', 'CODIGO_UNICO']].drop_duplicates()
contrato['IDCONT_'] = 1
contrato = contrato.pivot(index=['SEC_EJEC', 'CODIGO_UNICO'], columns='PERIODO_SEACE', values=['IDCONT_']).reset_index().rename_axis(None)
contrato.columns = [f'{i}{j}' for i, j in contrato.columns]

convocat = bd_convoca.copy()
convocat['SEC_EJEC'] = convocat['SEC_EJEC'].astype(int).astype(str)
convocat['CODIGO_UNICO'] = convocat['CODIGO_UNICO'].astype(int).astype(str)
convocat = convocat[['PERIODO_SEACE', 'SEC_EJEC', 'CODIGO_UNICO']].drop_duplicates()
convocat['IDCONV_'] = 1
convocat = convocat.pivot(index=['SEC_EJEC', 'CODIGO_UNICO'], columns='PERIODO_SEACE', values=['IDCONV_']).reset_index().rename_axis(None)
convocat.columns = [f'{i}{j}' for i, j in convocat.columns]

convcont = pd.merge(contrato, convocat, on=['SEC_EJEC', 'CODIGO_UNICO'], how='outer', validate='1:1')

In [6]:
#%% --------------------------------------------------------------------------- Compilación de bases
basef = siaf_ing.merge(siaf, on=['SEC_EJEC'], how='left', validate='1:m', indicator=True)
basef.rename(columns={'_merge':'ID_INGRESOS'}, inplace=True)
basef['ID_INGRESOS'] = basef['ID_INGRESOS'].replace({'left_only':'SI', 'both':'SI', 'right_only':'NO'})

basef = basef.merge(invpe, on=['CODIGO_UNICO'], how='left', validate='m:1', indicator=False)

basef = basef.merge(contrata2, on=['CODIGO_UNICO', 'SEC_EJEC'], how='left', validate='m:1', indicator=False)

basef = basef.merge(convcont, on=['CODIGO_UNICO', 'SEC_EJEC'], how='left', validate='m:1', indicator=False)

In [7]:

#%% --------------------------------------------------------------------------- Indicadores de base crítica

# Con marco por Rubro 18
basef['PIA_CON_R18'] = np.where(basef['PIA_2024R18'].fillna(0)>0, 'SI', 'NO')
# Con PIM por rubro 18 en el 2024
basef['PIM_CON_R18'] = np.where(basef['PIM_2024R18'].fillna(0)>0, 'SI', 'NO')
# Con PIM por rubro 18 en el 2023
basef['PIM23_CON_R18'] = np.where(basef['PIM_2023R18'].fillna(0)>0, 'SI', 'NO')

# Con compromisos lanzados en el 2023 según SIAF
basef['COMPROM_2023_CUIID'] = basef.groupby(['CODIGO_UNICO', 'SEC_EJEC'])['COMPROM_2023'].transform('sum')
basef['PIM_2023_CUIID'] = basef.groupby(['CODIGO_UNICO', 'SEC_EJEC'])['PIM_2023'].transform('sum')
basef['ConComprom_2023'] = np.where(basef['COMPROM_2023_CUIID'].fillna(0) > 0, 'SI', 'NO')
basef['ConPIM_2023'] = np.where(basef['PIM_2023_CUIID'].fillna(0) > 0, 'SI', 'NO')

# Con certificación en el 2023 según SIAF 
basef['CERTIF_2023_CUIID'] = basef.groupby(['CODIGO_UNICO', 'SEC_EJEC'])['CERTIF_2023'].transform('sum')
basef['ConCertif_2023'] = np.where(basef['CERTIF_2023_CUIID'].fillna(0) > 0, 'SI', 'NO')

# Rango de avance
basef['PIA_2024_CUIID'] = basef.groupby(['CODIGO_UNICO', 'SEC_EJEC'])['PIA_2024'].transform('sum')
basef['PIM_2024_CUIID'] = basef.groupby(['CODIGO_UNICO', 'SEC_EJEC'])['PIM_2024'].transform('sum')
basef['AVANCE_FINANCIERO_2024'] = basef[['DEV_ACUM_AL2023', 'PIM_2024_CUIID']].sum(axis=1) 
basef['%AVANCE_FINANCIERO_2024'] = np.where(basef['COSTO_ACTUAL_BCO'].fillna(0),
                                            basef['AVANCE_FINANCIERO_2024']/basef['COSTO_ACTUAL_BCO'], 
                                            np.nan)
basef['RANG_AVNC_FINCRO'] = np.where(((basef['%AVANCE_FINANCIERO_2024'] > 0)&(basef['%AVANCE_FINANCIERO_2024'] <= 0.1)), '<=10%', 
                                     np.where(((basef['%AVANCE_FINANCIERO_2024'] > 0.1)&(basef['%AVANCE_FINANCIERO_2024'] <= 0.3)), '10-30%', 
                                              np.where(((basef['%AVANCE_FINANCIERO_2024'] > 0.3)&(basef['%AVANCE_FINANCIERO_2024'] <= 0.5)), '30-50%',
                                                       np.where(((basef['%AVANCE_FINANCIERO_2024'] > 0.5)&(basef['%AVANCE_FINANCIERO_2024'] <= 0.7)), '50-70%', 
                                                                np.where(((basef['%AVANCE_FINANCIERO_2024'] > 0.7)&(basef['%AVANCE_FINANCIERO_2024'] <= 0.9)), '70-90%',
                                                                         np.where(((basef['%AVANCE_FINANCIERO_2024'] > 0.9)&(basef['%AVANCE_FINANCIERO_2024'].notna())), '>90%', ''))))))
basef['RANG_AVNC_FINCRO'] = np.where(basef['COSTO_ACTUAL_BCO'].fillna(0) == 0, 'SinCosto', basef['RANG_AVNC_FINCRO'])

# Pendiente por financiar
basef['PENDIENTE_X_FIN'] = np.where(basef['COSTO_ACTUAL_BCO'].fillna(0)>0,
                                    basef['COSTO_ACTUAL_BCO'] - basef['AVANCE_FINANCIERO_2024'], 
                                    0)

# Contrato o convocatoria SEACE
basef['ID_CONTRATO_2023'] = np.where((basef['IDCONT_22_23']==1)|(basef['IDCONT_23_24']==1), 'SI', 'NO')
basef['ID_CONVOCAT_2023'] = np.where((basef['IDCONV_23_24']==1), 'SI', 'NO')
basef['ID_CONV_CONT'] = np.where((basef['ID_CONTRATO_2023']=='SI')&(basef['ID_CONVOCAT_2023']=='SI'), 'Con contrato, con convocatoria', 
                                 np.where((basef['ID_CONTRATO_2023']=='SI')&(basef['ID_CONVOCAT_2023']!='SI'), 'Con contrato, sin convocatoria',
                                          np.where((basef['ID_CONTRATO_2023']!='SI')&(basef['ID_CONVOCAT_2023']=='SI'), 'Sin contrato, con convocatoria',
                                                   'Sin contrato, sin convocatoria')))

basef['INV_ESTATUS'] = np.where((basef['ESTADO']=='A')&
                                (basef['CERRADO']!='SI')&
                                (basef['SITUACION'].isin({'V', 'A'}))&
                                (basef['CODIGO_UNICO']!='2001621'), 'OK', 'NO')

# Escenario 1: más restrictivo
basef['ESCENARIO1'] = np.where((basef['INV_ESTATUS'].isin({'OK'}))& # INVERSIONES ACTIVAS
                               #((basef['PIA_CON_R18']=='SI'))& # CON MARCO EN EL PIA DE R18
                               (basef['EXP_TCO'].isin({'SI'}))& # CON EXPEDIENTE TÉCNICO VIGENTE
                               (basef['ConComprom_2023'].isin({'SI'}))& # CON COMPROMISOS REALIZADOS EN EL 2023
                               #(basef['%AVANCE_FINANCIERO_2024']>0.15)& # CON UN AVANCE FINANCIERO MAYOR AL 15%
                               (basef['ID_CONV_CONT'].isin({'Con contrato, con convocatoria', 'Con contrato, sin convocatoria'})) # CON CONTRATOS EN EL SEACE
                               #(basef['FINCONT_MAX_TOTAL'].fillna(0)>2023) # CON CONTRATOS VIGENTES MÁS ALLÁ DEL 2023
                               #(basef['PIA_2024_CUIID'].fillna(0)>0)& # CON MARCO INICIAL PARA EL 2024
                               #(basef['PIM_2024_CUIID'].fillna(0)>=basef['PIA_2024_CUIID'].fillna(0)), # CON MARCO PPTAL PARA EL 2024 NO REDUCIDO
                               ,'SI', 'NO')

# Escenario 2: medio
basef['ESCENARIO2'] = np.where((basef['INV_ESTATUS'].isin({'OK'}))& # INVERSIONES ACTIVAS
                               #((basef['PIA_CON_R18']=='SI'))& # CON MARCO EN EL PIA DE R18
                               (basef['EXP_TCO'].isin({'SI'}))& # CON EXPEDIENTE TÉCNICO VIGENTE
                               (basef['ConComprom_2023'].isin({'SI'})) # CON COMPROMISOS REALIZADOS EN EL 2023
                               #(basef['%AVANCE_FINANCIERO_2024']>0.15)& # CON UN AVANCE FINANCIERO MAYOR AL 15%
                               #(basef['ID_CONV_CONT'].isin({'Con contrato, con convocatoria', 'Con contrato, sin convocatoria'})) # CON CONTRATOS EN EL SEACE
                               #(basef['FINCONT_MAX_TOTAL'].fillna(0)>2023)& # CON CONTRATOS VIGENTES MÁS ALLÁ DEL 2023
                               #(basef['PIA_2024_CUIID'].fillna(0)>0)& # CON MARCO INICIAL PARA EL 2024
                               #(basef['PIM_2024_CUIID'].fillna(0)>=basef['PIA_2024_CUIID'].fillna(0)), # CON MARCO PPTAL PARA EL 2024 NO REDUCIDO
                               ,'SI', 'NO')

#------------------- despriorizar las notas que han sido catalogadas

# Escenario 3: asumiendo que ya hicieron reorientaciones
basef['ESCENARIO3'] = np.where((basef['INV_ESTATUS'].isin({'OK'}))& # INVERSIONES ACTIVAS
                               #((basef['PIA_CON_R18']=='SI'))& # CON MARCO EN EL PIA DE R18
                               (basef['EXP_TCO'].isin({'SI'}))& # CON EXPEDIENTE TÉCNICO VIGENTE
                               (basef['ConPIM_2023'].isin({'SI'})) # CON COMPROMISOS REALIZADOS EN EL 2023
                               #(basef['%AVANCE_FINANCIERO_2024']>0.15) # CON UN AVANCE FINANCIERO MAYOR AL 15%
                               #(basef['ID_CONV_CONT'].isin({'Con contrato, con convocatoria', 'Con contrato, sin convocatoria'}))& # CON CONTRATOS EN EL SEACE
                               #(basef['FINCONT_MAX_TOTAL'].fillna(0)>2023)& # CON CONTRATOS VIGENTES MÁS ALLÁ DEL 2023
                               #(basef['PIA_2024_CUIID'].fillna(0)>0)& # CON MARCO INICIAL PARA EL 2024
                               #(basef['PIM_2024_CUIID'].fillna(0)>=basef['PIA_2024_CUIID'].fillna(0)), # CON MARCO PPTAL PARA EL 2024 NO REDUCIDO
                               ,'SI', 'NO')

# Escenario 3: asumiendo que ya hicieron reorientaciones
basef['ESCENARIO4'] = np.where((basef['INV_ESTATUS'].isin({'OK'}))& # INVERSIONES ACTIVAS
                               #((basef['PIA_CON_R18']=='SI')|(basef['PIM_CON_R18']=='SI'))& # CON MARCO EN EL PIA DE R18
                               (basef['EXP_TCO'].isin({'SI'})) # CON EXPEDIENTE TÉCNICO VIGENTE
                               #(basef['ConPIM_2023'].isin({'SI'})) # CON COMPROMISOS REALIZADOS EN EL 2023
                               #(basef['%AVANCE_FINANCIERO_2024']>0.15) # CON UN AVANCE FINANCIERO MAYOR AL 15%
                               #(basef['ID_CONV_CONT'].isin({'Con contrato, con convocatoria', 'Con contrato, sin convocatoria'}))& # CON CONTRATOS EN EL SEACE
                               #(basef['FINCONT_MAX_TOTAL'].fillna(0)>2023)& # CON CONTRATOS VIGENTES MÁS ALLÁ DEL 2023
                               #(basef['PIA_2024_CUIID'].fillna(0)>0)& # CON MARCO INICIAL PARA EL 2024
                               #(basef['PIM_2024_CUIID'].fillna(0)>=basef['PIA_2024_CUIID'].fillna(0)), # CON MARCO PPTAL PARA EL 2024 NO REDUCIDO
                               ,'SI', 'NO')

basef['PPTO_ESCENARIO1'] = np.where((basef['ESCENARIO1']=='SI')&(basef['ID_INGRESOS']=='SI'), basef['PIA_2024R18'], 0)
basef['PPTO_ESCENARIO2'] = np.where((basef['ESCENARIO2']=='SI')&(basef['ID_INGRESOS']=='SI'), basef['PIA_2024R18'], 0)
basef['PPTO_ESCENARIO3'] = np.where((basef['ESCENARIO3']=='SI')&(basef['ID_INGRESOS']=='SI'), basef['PIA_2024R18'], 0)


print(basef['PPTO_ESCENARIO1'].sum())
print(basef['PPTO_ESCENARIO2'].sum())
print(basef['PPTO_ESCENARIO3'].sum())

#%%
##### Topeado por la variación
basef['TOPE_DIF'] = basef['NUEVO_INGRESO_R18']

for n in list(range(1,4)):
    basef['PPTO_ESCENARIO' + str(n) + '_Acum'] = basef.groupby(['UBIGEO'])['PPTO_ESCENARIO' + str(n)].transform('sum')
    basef['DIF_ESCENARIO' + str(n) + '_Acum'] = np.where(basef['ID_INGRESOS']=='SI',
                                                         basef['TOPE_DIF'] - basef['PPTO_ESCENARIO' + str(n) + '_Acum'],
                                                         0)
    # SI ES POSITIVA LA DIFERENCIA ES PORQUE EL "RECURSO POR CUBRIR" ES MAYOR QUE LA DIFERENCIA DE LA RD. POR LO TANTO
    # HABRIA QUE AJUSTAR A LA BAJA (REDUCIRLE AL RECURSO POR CUBRIR) Y VICEVERSA
    
    # CUBIERTOS -> Inversiones que tienen al menos 1 sol de PIM crítico. Proporción respecto a todo el PIM crítico
    basef['CUBIERTOS' + str(n)] = basef['PPTO_ESCENARIO' + str(n)]/basef['PPTO_ESCENARIO' + str(n) + '_Acum']
    basef['PPTO_ESCENARIO' + str(n) + '_AJUST'] = np.where(basef['DIF_ESCENARIO' + str(n) + '_Acum']<0, # solo cuando supera castigamos, caso contrario, se mantiene el monto
                                               basef['PPTO_ESCENARIO' + str(n)],
                                               0)

print(basef[(basef['ESCENARIO1']=='SI')&(basef['ID_INGRESOS']=='SI')]['PIM_2024R18'].sum())
print(basef[(basef['ESCENARIO2']=='SI')&(basef['ID_INGRESOS']=='SI')]['PIM_2024R18'].sum())
print(basef[(basef['ESCENARIO3']=='SI')&(basef['ID_INGRESOS']=='SI')]['PIM_2024R18'].sum())

print(basef['PPTO_ESCENARIO1'].sum())
print(basef['PPTO_ESCENARIO2'].sum())
print(basef['PPTO_ESCENARIO3'].sum())

basef_rev = basef.copy()
basef_rev = basef_rev.sort_values(by=['UBIGEO', 'PIA_2024', 'PIM_2024'], ascending=[True, True, True])
basef_rev = basef_rev[basef_rev['UBIGEO']!='']
basef_rev = basef_rev[basef_rev['ID_INGRESOS'] == 'SI']
basef_rev['ppto24'] = basef_rev[['PIA_2024', 'PIM_2024']].sum(axis=1)
basef_rev = basef_rev[basef_rev['ppto24']>0]

basef_rev = basef_rev[['UBIGEO', 'SECTOR', 'PLIEGO', 'EJECUTORA', 'PRODUCTO_PROYECTO', 
                       'PIA_2024', 'PIM_2024', 'PIA_2024R18', 'PIM_2024R18',
                       'NUEVO_INGRESO_R18', 
                       'ESTADO', 'CERRADO', 'SITUACION',
                       'PIA_CON_R18', 'ConComprom_2023', 'COMPROM_2023', 'ConPIM_2023', 'PIM_2023', 
                       'EXP_TCO', 'ET_VIGENTE', 'ID_CONV_CONT',
                       'ESCENARIO1', 'ESCENARIO2', 'ESCENARIO3',
                       'PPTO_ESCENARIO1', 'PPTO_ESCENARIO2', 'PPTO_ESCENARIO3',
                       'PPTO_ESCENARIO1_AJUST', 'PPTO_ESCENARIO2_AJUST', 'PPTO_ESCENARIO3_AJUST',
                       'PPTO_ESCENARIO1_Acum', 'PPTO_ESCENARIO2_Acum', 'PPTO_ESCENARIO3_Acum',]]

basef_rev.rename(columns={'NUEVO_INGRESO_R18':'NUEVO_INGRESO_R18_referencial',
                          'ESCENARIO1':'ESCENARIO1_CUI', 'ESCENARIO2':'ESCENARIO2_CUI', 'ESCENARIO3':'ESCENARIO3_CUI',
                          }, inplace=True)

print(basef_rev['PPTO_ESCENARIO1'].sum())
print(basef_rev['PPTO_ESCENARIO2'].sum())
print(basef_rev['PPTO_ESCENARIO3'].sum())


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [8]:

#%%
baseanterior = bd_mod_ant.copy()
baseanterior['CODIGO_UNICO'] = baseanterior['PRODUCTO_PROYECTO'].str.split('. ',n=1).str[0]
baseanterior['SEC_EJEC'] = baseanterior['SEC_EJEC'].astype(int).astype(str)
baseanterior = baseanterior[['CODIGO_UNICO', 'SEC_EJEC', 'ESCENARIO_SINPIM2023', 'ID_REDUC_MARCO', 'REDUCCION_MARCO']]
baseanterior = baseanterior[(baseanterior['ID_REDUC_MARCO'] == 'SI')&(baseanterior['ESCENARIO_SINPIM2023']=='SI')]
baseanterior = baseanterior.groupby(['SEC_EJEC', 'CODIGO_UNICO']).agg({'REDUCCION_MARCO':'sum'}).reset_index()
baseanterior.rename(columns={'REDUCCION_MARCO':'MONTOCALC_01FEB'}, inplace=True)
baseanterior['ID_CALC_ANTERIOR'] = 'SI'

basef_alt = basef.copy()
print(basef_alt.duplicated(['CODIGO_UNICO', 'SEC_EJEC']).sum())
basef_alt['dup'] = 1
basef_alt['dup'] = basef_alt.groupby(['SEC_EJEC', 'CODIGO_UNICO'])['dup'].transform('count')
basef_alt = basef_alt.sort_values(by=['dup', 'SEC_EJEC', 'CODIGO_UNICO'], ascending=[False, True, True])

basef_alt = basef_alt.merge(bajomarco_0102, on=['CODIGO_UNICO', 'SEC_EJEC'], how='left', validate='1:1')
basef_alt.rename(columns={'REDUCCION_MARCO':'REDUCCION_MARCO_0102', 'ID_RD31_MODIF':'ID_RD31_MODIF_0102'}, inplace=True)
basef_alt['ID_REDUC_MARCO_0102'] = np.where(basef_alt['REDUCCION_MARCO_0102']>0, 'SI', 'NO')
basef_alt['ID_APLICA_0102'] = np.where((basef_alt['ESCENARIO4']=='SI')&(basef_alt['ID_REDUC_MARCO_0102']=='SI'), 'SI', 'NO')

basef_alt = basef_alt.merge(bajomarco, on=['CODIGO_UNICO', 'SEC_EJEC'], how='left', validate='1:1')
basef_alt.rename(columns={'REDUCCION_MARCO':'REDUCCION_MARCO_0702', 'ID_RD31_MODIF':'ID_RD31_MODIF_0702'}, inplace=True)
basef_alt['ID_REDUC_MARCO_0702'] = np.where(basef_alt['REDUCCION_MARCO_0702']>0, 'SI', 'NO')
basef_alt['ID_APLICA_0702'] = np.where((basef_alt['ESCENARIO4']=='SI')&(basef_alt['ID_REDUC_MARCO_0702']=='SI'), 'SI', 'NO')


basef_alt.drop(columns=['PIA_2022OTR', 'PIA_2022R18', 'PIA_2022RO', 'PIM_2022OTR', 'PIM_2022R18', 'PIM_2022RO', 
                        'CERTIF_2022OTR', 'CERTIF_2022R18', 'CERTIF_2022RO', 'COMPROM_2022OTR', 'COMPROM_2022R18',
                        'COMPROM_2022RO', 'DEVENG_2022OTR', 'DEVENG_2022R18', 'DEVENG_2022RO',
                        'PIA_2023OTR', 'PIA_2023R18', 'PIA_2023RO',
                        'PIA_2022', 'PIM_2022', 'CERTIF_2022', 'COMPROM_2022', 'DEVENG_2022',
                        'FINCONT_22_23-CONSULT', 'FINCONT_22_23-NOINFO', 'FINCONT_22_23-OTROS',
                        'FINCONT_23_24-CONSULT', 'FINCONT_23_24-NOINFO', 'FINCONT_23_24-OTROS', 
                        ], inplace=True)

basef_alt = basef_alt[(basef_alt['PIA_2024'].fillna(0)>0)|(basef_alt['PIM_2024'].fillna(0)>0)]

print(basef_alt[(basef_alt['ESCENARIO1']=='SI')&(basef_alt['ID_INGRESOS']=='SI')]['PIM_2024R18'].sum())
print(basef_alt[(basef_alt['ESCENARIO2']=='SI')&(basef_alt['ID_INGRESOS']=='SI')]['PIM_2024R18'].sum())
print(basef_alt[(basef_alt['ESCENARIO3']=='SI')&(basef_alt['ID_INGRESOS']=='SI')]['PIM_2024R18'].sum())

basef_alt = secejec.merge(basef_alt, on='SEC_EJEC', how='right', validate='1:m')
basef_alt = postmerge(basef_alt)



basef_alt.rename(columns={'ESCENARIO2':'ESCENARIO_COMPROMISO2023', 'ESCENARIO4':'ESCENARIO_SINPIM2023'}, inplace=True)

basef_alt['ID_CALC_NUEVO'] = np.where((basef_alt['ESCENARIO_SINPIM2023'] == 'SI')&(basef_alt['ID_REDUC_MARCO_0702']=='SI'),
                                      'SI', 'NO')

basef_alt['IDCALCNUEVO+NOTAOK31'] = np.where((basef_alt['ID_CALC_NUEVO'] == 'SI')&(basef_alt['ID_RD31_MODIF_0702']=='SI'),
                                      'SI', 'NO')

basef_alt['MONTOCALC_07FEB'] = np.where(basef_alt['ID_CALC_NUEVO'] == 'SI', basef_alt['REDUCCION_MARCO_0702'], np.nan)
basef_alt['MONTOCALC_07FEB_FINAL'] = np.where(basef_alt['IDCALCNUEVO+NOTAOK31'] == 'SI', basef_alt['REDUCCION_MARCO_0702'], np.nan)

# Montos anteriores!
basef_alt = basef_alt.merge(baseanterior, on=['CODIGO_UNICO', 'SEC_EJEC'], how='outer', validate='1:1')
basef_alt['ID_CALC_ANTERIOR'] = basef_alt['ID_CALC_ANTERIOR'].fillna('NO')

print(basef_alt['MONTOCALC_01FEB'].sum())
print(basef_alt['MONTOCALC_07FEB'].sum())

basef_alt = basef_alt[['UBIGEO', 'DEPARTAMENTO', 'NIVEL_GOB', 'SECTOR', 'PLIEGO', 'EJECUTORA', 'PLIEGO_SIAF', 'PRODUCTO_PROYECTO', 'CODIGO_UNICO', 'SEC_EJEC',
                       'PIA_2024', 'PIM_2024', 'PIA_2024R18', 'PIM_2024R18', 'ID_INGRESOS', 
                       'NUEVO_INGRESO_R18', 'ID_RD31_MODIF_0102', 'ID_RD31_MODIF_0702',
                       'ESTADO', 'CERRADO', 'SITUACION',
                       'PIA_CON_R18', 'ConComprom_2023', 'COMPROM_2023', 'ConPIM_2023', 'PIM_2023', 
                       'EXP_TCO', 'ET_VIGENTE', 'ID_CONV_CONT', 'TIPO_PROYECTO',
                       'ESCENARIO_COMPROMISO2023', 'ESCENARIO_SINPIM2023',
                       #'ID_REDUC_MARCO', 'REDUCCION_MARCO', # DATOS DE LA CORRIDA AL 01.02
                       'ID_REDUC_MARCO_0102', 'REDUCCION_MARCO_0102', 
                       'ID_REDUC_MARCO_0702', 'REDUCCION_MARCO_0702', 
                       'ID_CALC_ANTERIOR', 'MONTOCALC_01FEB', 'ID_CALC_NUEVO', 'IDCALCNUEVO+NOTAOK31', 'MONTOCALC_07FEB', 'MONTOCALC_07FEB_FINAL', 
                       'Cuartil 1', 'CAIDA_RD031', 'ID_FECHAOK']]

#%% Datos adicionales al 12.02

0
0.0
0.0
0.0
359102367.0
430112116.0


In [9]:
basef_alt.head()

,UBIGEO,DEPARTAMENTO,NIVEL_GOB,SECTOR,PLIEGO,EJECUTORA,PLIEGO_SIAF,PRODUCTO_PROYECTO,CODIGO_UNICO,SEC_EJEC,...,REDUCCION_MARCO_0702,ID_CALC_ANTERIOR,MONTOCALC_01FEB,ID_CALC_NUEVO,IDCALCNUEVO+NOTAOK31,MONTOCALC_07FEB,MONTOCALC_07FEB_FINAL,Cuartil 1,CAIDA_RD031,ID_FECHAOK
0,150000,15. LIMA REGION,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,001. SEDE LIMA,99463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,2001621. ESTUDIOS DE PRE-INVERSION,2001621,1027,...,NaN,NO,NaN,NO,NO,NaN,NaN,75,-8640241.4,NaN
1,150000,15. LIMA REGION,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,001. SEDE LIMA,99463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,2016766. INICIATIVA A LA COMPETITIVIDAD,2016766,1027,...,NaN,NO,NaN,NO,NO,NaN,NaN,75,-8640241.4,NaN
2,150000,15. LIMA REGION,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,001. SEDE LIMA,99463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,2063107. AMPLIACION Y MEJORAMIENTO DE LOS SIST...,2063107,1027,...,NaN,NO,NaN,NO,NO,NaN,NaN,75,-8640241.4,NaN
3,150000,15. LIMA REGION,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,001. SEDE LIMA,99463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,"2089754. EXPEDIENTES TECNICOS, ESTUDIOS DE PRE...",2089754,1027,...,NaN,NO,NaN,NO,NO,NaN,NaN,75,-8640241.4,NaN
4,150000,15. LIMA REGION,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,001. SEDE LIMA,99463. GOBIERNO REGIONAL DEL DEPARTAMENTO DE LIMA,2106392. CONSTRUCCION Y EQUIPAMIENTO DEL HOSPI...,2106392,1027,...,NaN,NO,NaN,NO,NO,NaN,NaN,75,-8640241.4,NaN


In [10]:
#%%
outputFile = os.path.join(r'C:\Users\jchamba\Documents\Jannely\Proyecto 4\RD_031', 'BD_rd31eval_Vresum_10GR' + '2902024' + ".xlsx")
with pd.ExcelWriter(outputFile) as ew:
    basef_alt.to_excel(ew, sheet_name="DATA", index = False)
    
#%%